In [1]:
import dvc.api
import pandas as pd
import sys 
sys.path.append('../../')
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [4]:
from pytorch_lightning.utilities.deepspeed import convert_zero_checkpoint_to_fp32_state_dict
save_path = '/workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_0/epoch=4-step=4630.ckpt'
output_path = '/workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_0/epoch=4-step=4630.ckpt/pytorch_model.bin'


In [5]:
convert_zero_checkpoint_to_fp32_state_dict(save_path, output_path)

Processing zero checkpoint '/workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_0/epoch=4-step=4630.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 6
Parsing checkpoint created by deepspeed==0.7.3
Reconstructed fp32 state dict with 559 params 783150081 elements
Saving fp32 state dict to /workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_0/epoch=4-step=4630.ckpt/pytorch_model.bin


In [6]:
model = LLM_MultitaskMultimodal.load_from_checkpoint('/workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_0/epoch=4-step=4630.ckpt/pytorch_model.bin')

INFO:root:Unused kwargs when getting google/flan-t5-large: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'clm_singlemodal_wishtitledesp2attrkvpair': None}, 'head_dict': {}}


In [8]:
def format_input(title, description, category):
    template = f'[title start] {title} [title end] [description start] {description} [description end] [taxonomy start] {category} [taxonomy end]'
    task_prefix = 'Generate attribute key value pairs for product with description and taxonomy: '
    return task_prefix + template

In [12]:
df = pd.read_json(dvc.api.get_url( 
    'datasets/data/wish_attr_extract_label/processed/appen_020323_030323_delivered_030623_validated_product_attr_textonly_train.json', 
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [14]:
rec = df.sample(1).to_dict('records')[0]

In [17]:
input_text = format_input(rec['title'], rec['description'], rec['category'])

In [24]:
rec

{'label_ordering': 8576,
 'sample_method': 'only_text',
 'pid': '5ffddddf87dcf4d88684b371',
 'category': 'Sports > Fishing > Fishing Lures',
 'title': 'Vib Spoon Lure 15G 20G Metal Fishing Lures Sequins Spinner Vibration Hard Ice Bait Pesca Jigging Japan Tackle Winter',
 'description': 'Brand Name: Lqcrwoc\nPosition: Ocean Boat Fishing\nPosition: Ocean Beach Fishing\nPosition: Lake\nPosition: River\nPosition: Reservoir Pond\nPosition: Stream\nPosition: Ocean Rock Fishing\nCategory: Lure\nModel Number: Dmt2P\nType: Artificial Bait\nType: Vib Sequins Bait\nSize: 15G/20G\nMaterial: Metal\nColor: Gold ,  Silver',
 'main_img_url': nan,
 'rater_output_processed': 'Sports > Fishing > Fishing Lures > Primary Color > Gold\nSports > Fishing > Fishing Lures > Primary Color > Silver',
 'attr_name_value_pairs_normalized': [['Primary Color', 'Gold'],
  ['Primary Color', 'Silver']],
 'attr_name_value_pairs_custom': [],
 'attr_name_value_pairs_normalized_text': 'Primary Color|Gold\nPrimary Color|Silve

In [28]:
model.eval()
model.tokenizer.batch_decode(model.transformer.generate(**model.tokenizer(input_text, return_tensors='pt'), min_length=100, max_length=100))

['<pad> Materials|Metal [NL] Primary Color|Silver [NL] Styles|Sinking [NL] Styles|Sinking [NL] Styles|Sintage [NL] Styles|Sintage [NL] Styles|Sintage [NL] Styles|Sintage [NL] Styles|Sintage [NL] Styles|Sintage [NL] Styles|Sintage [NL] Styles']